In [4]:
import os
import pandas as pd

In [3]:
# Define the directories and file paths
port_directory = r"C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\1_Input\05_ConstructPort\05_Stock_MonthYear_Port"
stock_file_path = r"C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\1_Input\05_ConstructPort\04_MergedStockFile\AllMeragedStock.csv"
riskfree_data_path = r"C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\1_Input\05_ConstructPort\00_RiskFree\00_Riskfree_2012_2024_Lag1.csv"
output_directory = r"C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\05_Port_Month_Year_FullData"

In [5]:
# Read the stock data and risk-free rate data into DataFrames
stock_data = pd.read_csv(stock_file_path)
riskfree_data = pd.read_csv(riskfree_data_path)


In [9]:
# Convert 'Month_Year' to datetime format, setting it to the start of each month
stock_data['Month_Year'] = pd.to_datetime(stock_data['Month_Year'], format='%Y-%m')
riskfree_data['Month_Year'] = pd.to_datetime(riskfree_data['Month_Year'], format='%Y-%m')


In [14]:
# Loop through each CSV file in the port directory
for filename in os.listdir(port_directory):
    if filename.endswith(".csv"):
        # Construct the full path to the portfolio file
        port_file_path = os.path.join(port_directory, filename)
        
        # Read the portfolio data into a DataFrame
        port_data = pd.read_csv(port_file_path)
        
        # Strip any potential whitespace from column names and convert 'Month_Year' to datetime format
        port_data.columns = port_data.columns.str.strip()
        port_data['Month_Year'] = pd.to_datetime(port_data['Month_Year'], format='%Y-%m')
        
        # Merge portfolio data with stock data on 'Month_Year' and 'Name_stock'
        merged_df = pd.merge(port_data, stock_data[["Month_Year", "Name_stock", "volatility_stock", 
                                                    "3year_rolling_correlation", "Adjusted_Beta", "ROI_Lag1"]], 
                             how='left', on=['Month_Year', 'Name_stock'])
        
        # Merge the result with risk-free data on 'Month_Year'
        final_merged_df = pd.merge(merged_df, riskfree_data[["Month_Year","RF_Lag1"]], how='left', on='Month_Year')

        final_merged_df['Month_Year'] = final_merged_df['Month_Year'].dt.to_period('M')


        # Drop rows with NaN in the 'Adjusted_Beta' column
        final_merged_df.dropna(subset=['Adjusted_Beta'], inplace=True)
        
        # Construct the output file path using the same filename
        output_file_path = os.path.join(output_directory, filename)
        
        # Save the final merged DataFrame to CSV in the output directory
        final_merged_df.to_csv(output_file_path, index=False)
        
        print(f"Processed and saved merged data for file: {filename} to {output_file_path}")

Processed and saved merged data for file: PORT_2009_1.csv to C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\05_Port_Month_Year_FullData\PORT_2009_1.csv
Processed and saved merged data for file: PORT_2009_10.csv to C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\05_Port_Month_Year_FullData\PORT_2009_10.csv
Processed and saved merged data for file: PORT_2009_11.csv to C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\05_Port_Month_Year_FullData\PORT_2009_11.csv
Processed and saved merged data for file: PORT_2009_12.csv to C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\05_Port_Month_Year_FullData\PORT_2009_12.csv
Processed and saved merged data for file: PORT_2009_2.csv to C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\05_Port_Month_Year_FullData\PORT_2009_2.csv
Processed and saved merged data for file: PORT_2009_3.csv to C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\2_Output\05_Port_Month_Year_FullData\PORT_2009_3.csv
Processed and saved me

In [8]:
print("Port Data Columns: ", port_data.columns)
print("Stock Data Columns: ", stock_data.columns)
print("Riskfree Data Columns: ", riskfree_data.columns)


Port Data Columns:  Index(['Month_Year', 'Name_stock'], dtype='object')
Stock Data Columns:  Index(['Month_Year', 'Name_stock', 'Close_stock', 'volatility_stock',
       '3year_rolling_correlation', 'Adjusted_Beta', 'pct_ROI', 'ROI_Lag1'],
      dtype='object')
Riskfree Data Columns:  Index(['Month_Year', 'Date', 'T-BILL1M', 'DecimalM_T-BILL1M', 'RF_Lag1'], dtype='object')
